In [1]:
import sympy as sp
import numpy as np
from numpy.linalg import solve

L_pp, L_ss, k_a, k_b, u_e, omega = sp.symbols("L_{pp}, L_{ss}, k_a, k_b, u_e, \omega", real=True)

n = 5
A_ = sp.MatrixSymbol("\mathbf{A}", 2*n, 2*n)
L_ = sp.MatrixSymbol("\mathbf{L}", n, n)
E1_ = sp.MatrixSymbol("\mathbf{E_1}", n, n)
E2_ = sp.MatrixSymbol("\mathbf{E_2}", n, n)
e_ = sp.MatrixSymbol("\mathbf{e}", n, 1)
b_ = sp.MatrixSymbol("\mathbf{b}", 2*n, 1)
u_ = sp.MatrixSymbol("\mathbf{u}", n, 1)
i_ = sp.MatrixSymbol("\mathbf{i}", n, 1)
x_ = sp.MatrixSymbol("\mathbf{x}", 2*n, 1)


# IT 8.10/IT 8.11 Impedanzmatritzen

Bei einem Mehrwicklungstrafo benötigt man einige Parameter, um das Verhalten zu bestimmen.

Bei Messungen konnten einige Symmetrien erkannt werden, um das System zu beschreiben, werden folgende Parameter verwendet:

  * $L_{pp}$: Die Selbstinduktivität einer Primärwicklung
  * $L_{ss}$: Die Selbstinduktivität der Sekundärwicklung
  * $k_a$: Der Koppelfaktor zwischen zwei nebeneinander liegenden Primärwicklungen
  * $k_b$: Der Koppelfaktor zwischen einer Primär- und der Sekundärwicklung.
  
Kopplungen zwischen entfernter liegenden Primärwicklungen sind sehr klein und werden ignoriert.

Mit 

$$
M_{ij} = k_{ij}\sqrt{L_{ii} L_{jj}}
$$

erhält man als Systemgleichung:

$$
\begin{pmatrix}
  u_1 \\
  u_2 \\
  u_3 \\
  u_4 \\
  u_s
\end{pmatrix} = j\omega
\begin{pmatrix}
  L_{pp}                  & k_a L_{pp}              & 0                       & 0                       & k_b\sqrt{L_{pp} L_{ss}} \\
  k_a L_{pp}              & L_{pp}                  & k_a L_{pp}              & 0                       & k_b\sqrt{L_{pp} L_{ss}} \\
  0                       & k_a L_{pp}              & L_{pp}                  & k_a L_{pp}              & k_b\sqrt{L_{pp} L_{ss}} \\
  0                       & 0                       & k_a L_{pp}              & L_{pp}                  & k_b\sqrt{L_{pp} L_{ss}} \\
  k_b\sqrt{L_{pp} L_{ss}} & k_b\sqrt{L_{pp} L_{ss}} & k_b\sqrt{L_{pp} L_{ss}} & k_b\sqrt{L_{pp} L_{ss}} & L_{ss} \\
\end{pmatrix}
\begin{pmatrix}
  i_1 \\
  i_2 \\
  i_3 \\
  i_4 \\
  i_s
\end{pmatrix}
$$

In [2]:
L = sp.Matrix([
  [L_pp, k_a*L_pp, 0, 0, k_b*sp.sqrt(L_pp*L_ss)],
  [k_a*L_pp, L_pp, k_a*L_pp, 0, k_b*sp.sqrt(L_pp*L_ss)],
  [0, k_a*L_pp, L_pp, k_a*L_pp, k_b*sp.sqrt(L_pp*L_ss)],
  [0, 0, k_a*L_pp, L_pp, k_b*sp.sqrt(L_pp*L_ss)],
  [k_b*sp.sqrt(L_pp*L_ss), k_b*sp.sqrt(L_pp*L_ss), k_b*sp.sqrt(L_pp*L_ss), k_b*sp.sqrt(L_pp*L_ss), L_ss]
])

Um aus dieser Matrix nun Werte abzuleiten, muss man das System in Tableaugleichungen überführen:

$$
    \mathbf{0} = \begin{pmatrix}\mathbf{1} & -j\omega \mathbf{L} \\\mathbf{E_1} & \mathbf{E_2}\end{pmatrix} \cdot \begin{pmatrix}\mathbf{u}\\\mathbf{i}\end{pmatrix} - \begin{pmatrix}\mathbf{0} \\\mathbf{e}\end{pmatrix}\text{,}
$$

wobei mit $\mathbf{E_1}$, $\mathbf{E_2}$ und $\mathbf{e}$ die Erregung beschrieben werden kann.

Für Leerlauf bei allen Ports (Strom $i$ ist $0$) wäre beispielsweise $\mathbf{E_2}$ die Einheitsmatrix und $\mathbf{e}_i = 0$.

Einfachheitshalber werden im folgenden die Spannungen mit $u' = u/(j\omega)$ skaliert, dadurch wird das System leichter:

$$
    \mathbf{0} = \begin{pmatrix}\mathbf{1} & -\mathbf{L} \\\mathbf{E_1} & \mathbf{E_2}\end{pmatrix} \cdot \begin{pmatrix}\mathbf{u'}\\\mathbf{i}\end{pmatrix} - \begin{pmatrix}\mathbf{0} \\\mathbf{e'}\end{pmatrix}\text{,}
$$

Bei der Erregung $e$ muss man jeweils auch darauf achten. Dieser Ansatz funktioniert nur, solange nur frequenzabhängige Elemente vorhanden sind. Wären ohmsche Widerstände dabei, würde diese Normierung das System komplizierter machen.

## Kurzschlussinduktivität bei paralleler Einspeisung

In [3]:
E1 = sp.Matrix([
    [1,  0, 0, 0, 0],
    [1, -1, 0, 0, 0],
    [1,  0, -1, 0, 0],
    [1,  0, 0, -1, 0],
    [0,  0, 0, 0, 1],
])

display(sp.Eq(E1_, E1))

E2 = sp.Matrix([
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
])

display(sp.Eq(E2_, E2))

e = sp.Matrix([u_e, 0, 0, 0, 0])
display(sp.Eq(e_, e))

Eq(\mathbf{E_1}, Matrix([
[1,  0,  0,  0, 0],
[1, -1,  0,  0, 0],
[1,  0, -1,  0, 0],
[1,  0,  0, -1, 0],
[0,  0,  0,  0, 1]]))

Eq(\mathbf{E_2}, Matrix([
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0]]))

Eq(\mathbf{e}, Matrix([
[u_e],
[  0],
[  0],
[  0],
[  0]]))

Man erhält so ein lineares Gleichungssystem

$$
0 = \mathbf{Ax - b}
$$

mit

In [4]:
A = sp.BlockMatrix([[sp.Identity(n), -L], [E1, E2]])
display(sp.Eq(A_, A.as_explicit()))
b = sp.BlockMatrix([[sp.zeros(n, 1)], [e]])
display(sp.Eq(b_, b.as_explicit()))

Eq(\mathbf{A}, Matrix([
[1,  0,  0,  0, 0,                  -L_{pp},              -L_{pp}*k_a,                        0,                        0, -k_b*sqrt(L_{pp}*L_{ss})],
[0,  1,  0,  0, 0,              -L_{pp}*k_a,                  -L_{pp},              -L_{pp}*k_a,                        0, -k_b*sqrt(L_{pp}*L_{ss})],
[0,  0,  1,  0, 0,                        0,              -L_{pp}*k_a,                  -L_{pp},              -L_{pp}*k_a, -k_b*sqrt(L_{pp}*L_{ss})],
[0,  0,  0,  1, 0,                        0,                        0,              -L_{pp}*k_a,                  -L_{pp}, -k_b*sqrt(L_{pp}*L_{ss})],
[0,  0,  0,  0, 1, -k_b*sqrt(L_{pp}*L_{ss}), -k_b*sqrt(L_{pp}*L_{ss}), -k_b*sqrt(L_{pp}*L_{ss}), -k_b*sqrt(L_{pp}*L_{ss}),                  -L_{ss}],
[1,  0,  0,  0, 0,                        0,                        0,                        0,                        0,                        0],
[1, -1,  0,  0, 0,                        0,                        0,      

Eq(\mathbf{b}, Matrix([
[  0],
[  0],
[  0],
[  0],
[  0],
[u_e],
[  0],
[  0],
[  0],
[  0]]))

Die Lösung dieses Systems ist

In [5]:
solution = A.as_explicit().solve(b.as_explicit())
display(sp.Eq(x_, solution.simplify()))

Eq(\mathbf{x}, Matrix([
[                                                                                                  u_e],
[                                                                                                  u_e],
[                                                                                                  u_e],
[                                                                                                  u_e],
[                                                                                                    0],
[                                           u_e/(L_{pp}*(-k_a**2 + 2*k_a*k_b**2 + k_a - 4*k_b**2 + 1))],
[                                 u_e*(1 - k_a)/(L_{pp}*(-k_a**2 + 2*k_a*k_b**2 + k_a - 4*k_b**2 + 1))],
[                                 u_e*(1 - k_a)/(L_{pp}*(-k_a**2 + 2*k_a*k_b**2 + k_a - 4*k_b**2 + 1))],
[                                           u_e/(L_{pp}*(-k_a**2 + 2*k_a*k_b**2 + k_a - 4*k_b**2 + 1))],
[2*k_b*u_e*sqrt(L_{pp}*L_{ss})*

Woraus man nun die Primärseitige Kurzschlussinduktivität bei Parallelschaltung berechnen kann:

In [6]:
L_k1_ = sp.symbols("L_{k4\parallel}")

L_k1 = solution[0]/(sum(solution[5:9]))

eq = sp.Eq(L_k1_, L_k1.simplify())
display(eq)

Eq(L_{k4\parallel}, L_{pp}*(k_a**2 - 2*k_a*k_b**2 - k_a + 4*k_b**2 - 1)/(2*(k_a - 2)))

Als erstes soll der Grenzfall $k_a\rightarrow0$ betrachtet werden. Die gemessene Induktivität wird dann

In [7]:
L_k1.subs({k_a: 0.0}).simplify()

L_{pp}*(1/4 - k_b**2)

Für eine verschwindende Eingangsimpedanz müsste also $k_b = \frac{1}{2}$ sein. Das entspricht auch den Erwartungen für einen solchen Aufbau bei perfekter Kopplung:

  * Der Flusskoppelfaktor von einer Primärspule zur Sekundärspule ist $k_{ps} = 1$, da der gesamte Fluss, der durch die Primärspule geht, auch durch die Sekundärspule geht.
  * Der Flusskoppelfaktor in die andere Richtung beträgt $k_{sp} = \frac{1}{4}$: Ein Viertel des Flusses der Sekundärspule geht durch eine Primärspule
  
Der kombinierte Koppelfaktor ist somit:

$$
k = \sqrt{k_{sp}k_{ps}} = \sqrt{1*\frac{1}{4}} = \frac{1}{2}
$$

Setzt man nun $k_a > 0$, z.B. $k_a = 1\%$, wird die gemessene Induktivität

In [18]:
L_k1.subs({k_a: 0.01, k_b: 0.5}).simplify()

0.00374371859296482*L_{pp}

$k_b$ müsste also etwas größer werden, damit die Eingangsinduktivität wieder verschwindet.

## Eingangsinduktivität bei Kurzschluss aller anderer Wicklungen

In [9]:
L_k2_ = sp.symbols("L_{k2}")

E1 = sp.Matrix([
    [1,  0, 0, 0, 0],
    [0,  1, 0, 0, 0],
    [0,  0, 1, 0, 0],
    [0,  0, 0, 1, 0],
    [0,  0, 0, 0, 1],
])


E2 = sp.Matrix([
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
])

e = sp.Matrix([u_e, 0, 0, 0, 0])

A = sp.BlockMatrix([[sp.Identity(n), -L], [E1, E2]])
b = sp.BlockMatrix([[sp.zeros(n, 1)], [e]])
solution = A.as_explicit().solve(b.as_explicit())
display(sp.Eq(x_, solution.simplify()))


L_k2 = solution[0]/solution[5]
display(sp.Eq(L_k2_, L_k2.simplify()))

Eq(\mathbf{x}, Matrix([
[                                                                                                                                                      u_e],
[                                                                                                                                                        0],
[                                                                                                                                                        0],
[                                                                                                                                                        0],
[                                                                                                                                                        0],
[             u_e*(-2*k_a**2 + 4*k_a*k_b**2 - 3*k_b**2 + 1)/(L_{pp}*(k_a**4 - 2*k_a**3*k_b**2 + 2*k_a**2*k_b**2 - 3*k_a**2 + 6*k_a*k_b**2 - 4*k_b**2 + 1))],
[u_e*(k_a**3 - 2*k_a**2*k_b**2 + k

Eq(L_{k2}, L_{pp}*(-k_a**4 + 2*k_a**3*k_b**2 - 2*k_a**2*k_b**2 + 3*k_a**2 - 6*k_a*k_b**2 + 4*k_b**2 - 1)/(2*k_a**2 - 4*k_a*k_b**2 + 3*k_b**2 - 1))

Diese Induktivität kann man nun mit der vorherigen Induktivität vergleichen, bei der die vier Primärwicklungen parallel geschaltet sind:

In [10]:
ratio = (L_k2/L_k1).simplify()

display(sp.Eq(L_k2_/L_k1_, ratio))

Eq(L_{k2}/L_{k4\parallel}, 2*(-k_a**3 + k_a**2 + 3*k_a - 2)/(2*k_a**2 - 4*k_a*k_b**2 + 3*k_b**2 - 1))

Im Grenzfall $k_a \rightarrow 0$ ist dieses Verhältnis

In [11]:
ratio.subs({k_a: 0}).simplify()

-4/(3*k_b**2 - 1)

Was für perfekte Kopplung $k_b = \frac{1}{2}$

In [12]:
ratio.subs({k_a: 0, k_b: 1/2}).simplify()

16.0000000000000

ist.

Setzt man aber nun bspw. $k_a = 1\%$ und $k_b = 49\%$ ergibt sich ein kleineres Verhältnis:

In [19]:
ratio.subs({k_a: 0.01, k_b: 0.5}).simplify()

15.1647498075443

## Drei parallel geschaltete Wicklungen

In [14]:
L_k3_ = sp.symbols("L_{k3\parallel}")

E1 = sp.Matrix([
    [1,  0,  0, 0, 0],
    [1, -1,  0, 0, 0],
    [1,  0, -1, 0, 0],
    [0,  0,  0, 0, 0],
    [0,  0,  0, 0, 1],
])


E2 = sp.Matrix([
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0],
])

e = sp.Matrix([u_e, 0, 0, 0, 0])

A = sp.BlockMatrix([[sp.Identity(n), -L], [E1, E2]])
b = sp.BlockMatrix([[sp.zeros(n, 1)], [e]])
solution = A.as_explicit().solve(b.as_explicit())
display(sp.Eq(x_, solution.simplify()))


L_k3 = solution[0]/sum(solution[5:9])
display(sp.Eq(L_k3_, L_k3.simplify()))

Eq(\mathbf{x}, Matrix([
[                                                                                             u_e],
[                                                                                             u_e],
[                                                                                             u_e],
[           u_e*(k_a**2 - 4*k_a*k_b**2 - k_a + 3*k_b**2)/(2*k_a**2 - 4*k_a*k_b**2 + 3*k_b**2 - 1)],
[                                                                                               0],
[                                 u_e*(k_a - 1)/(L_{pp}*(2*k_a**2 - 4*k_a*k_b**2 + 3*k_b**2 - 1))],
[                               u_e*(2*k_a - 1)/(L_{pp}*(2*k_a**2 - 4*k_a*k_b**2 + 3*k_b**2 - 1))],
[                                 u_e*(k_a - 1)/(L_{pp}*(2*k_a**2 - 4*k_a*k_b**2 + 3*k_b**2 - 1))],
[                                                                                               0],
[k_b*u_e*sqrt(L_{pp}*L_{ss})*(3 - 4*k_a)/(L_{pp}*L_{ss}*(2*k_a**2 - 4*k_a*k_

Eq(L_{k3\parallel}, L_{pp}*(2*k_a**2 - 4*k_a*k_b**2 + 3*k_b**2 - 1)/(4*k_a - 3))

In [15]:
# Die gemessene Induktivität wird 0 für

L_k3.subs({k_a: 0, k_b: 1/np.sqrt(3)})

-1.11022302462516e-16*L_{pp}

## Zur Verifikation: In Reihe geschaltete Primäreseite

In [16]:
L_k4_ = sp.symbols("L_{k4}")

E1 = sp.Matrix([
    [1,  0,  0, 0, 0],
    [0,  0,  0, 0, 0],
    [0,  0,  0, 0, 0],
    [0,  0,  0, 0, 0],
    [0,  0,  0, 0, 1],
])


E2 = sp.Matrix([
    [0, 0, 0, 0, 0],
    [1, -1, 0, 0, 0],
    [1, 0, -1, 0, 0],
    [1, 0, 0, -1, 0],
    [0, 0, 0, 0, 0],
])

e = sp.Matrix([u_e, 0, 0, 0, 0])

A = sp.BlockMatrix([[sp.Identity(n), -L], [E1, E2]])
b = sp.BlockMatrix([[sp.zeros(n, 1)], [e]])
solution = A.as_explicit().solve(b.as_explicit())
display(sp.Eq(x_, solution.simplify()))


L_k4 = solution[0]/solution[5]
display(L_k4)
display(sp.Eq(L_k4_, L_k4.simplify()))

Eq(\mathbf{x}, Matrix([
[                                                                u_e],
[                    u_e*(2*k_a - 4*k_b**2 + 1)/(k_a - 4*k_b**2 + 1)],
[                    u_e*(2*k_a - 4*k_b**2 + 1)/(k_a - 4*k_b**2 + 1)],
[                                                                u_e],
[                                                                  0],
[                                  u_e/(L_{pp}*(k_a - 4*k_b**2 + 1))],
[                                  u_e/(L_{pp}*(k_a - 4*k_b**2 + 1))],
[                                  u_e/(L_{pp}*(k_a - 4*k_b**2 + 1))],
[                                  u_e/(L_{pp}*(k_a - 4*k_b**2 + 1))],
[-4*k_b*u_e*sqrt(L_{pp}*L_{ss})/(L_{pp}*L_{ss}*(k_a - 4*k_b**2 + 1))]]))

L_{pp}*k_a - 4*L_{pp}*k_b**2 + L_{pp}

Eq(L_{k4}, L_{pp}*(k_a - 4*k_b**2 + 1))